In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.2/434.2 MB 20.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-4.0.1-py2.py3-none-any.whl size=434813860 sha256=b727f649deebb71dbedc58b20a41ae2b809c89e79ab3a4412a032b4a6ea84625
  Stored in directory: /Users/maximesf/Library/Caches/pip/wheels/00/e3/92/8594f4cee2c9fd4ad82fe85e4bf2559ab8ea84ef19b1dd3d15
Successfully built pyspark

[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install mmh3



[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

zsh:1: command not found: apt-get
zsh:1: command not found: apt-get


In [4]:
# # Créer une session Spark
# spark = SparkSession.builder.appName("Exemple").getOrCreate()

# # Charger un fichier CSV
# df = spark.read.csv("metaverse_transactions_dataset.csv", header=True, inferSchema=True)

# # Afficher le schéma et une partie des données
# df.printSchema()
# df.show()



In [5]:
# type(df)

In [6]:
# df.select(['timestamp','amount']).show()

In [7]:
#spark.stop()

In [8]:
import pyspark
from pyspark.sql import DataFrame, SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col
import hashlib
from pyspark.sql.functions import lower, regexp_replace, split, udf
from pyspark.sql.types import ArrayType, StringType, LongType, IntegerType

Initialiser Apache Spark

In [9]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Document Similarity") \
    .getOrCreate()
sc = spark.sparkContext


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/06 11:30:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:588)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2446)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2446)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:339)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:59)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:485)
	a

In [ ]:
import mmh3

**Classe Shingling**

In [ ]:
class Shingling():
    def __init__(self, k=10):
        """
        k: length of shingles (in words)
        """
        self.k = k

    def clean_text(self, df: DataFrame, text_col: str) -> DataFrame:
        # Mettre en minuscule, supprimer caractères spéciaux, réduire espaces multiples
        df = df.withColumn(text_col, lower(col(text_col)))
        df = df.withColumn(text_col, regexp_replace(col(text_col), r"[^a-z0-9\s]", ""))
        df = df.withColumn(text_col, regexp_replace(col(text_col), r"\s+", " "))
        return df

    def tokenize(self, df: DataFrame, text_col: str) -> DataFrame:
        df = df.withColumn("tokens", split(col(text_col), "\\s+"))

        def clean_tokens(tokens):
            # Supprimer None et chaînes vides
            return [str(t).strip() for t in tokens if t and str(t).strip()]

        clean_udf = udf(clean_tokens, ArrayType(StringType())) #udf(...) transforme clean_tokens en fonction distribuée Spark.#Spark ne peut pas directement appliquer des fcts Python, besoin d'un wrapper -> udf
        df = df.withColumn("tokens", clean_udf(col("tokens")))
        return df

    def generate_shingles(self, df: DataFrame) -> DataFrame:
        k = self.k

        def shingle_function(tokens):
            tokens = [str(t).strip() for t in tokens if t and str(t).strip()]
            if len(tokens) < k:
                return []
            shingles = [" ".join(tokens[i:i+k]) for i in range(len(tokens) - k + 1)]
            # Supprimer shingles vides ou None
            return [s for s in shingles if s and s.strip()]

        shingle_udf = udf(shingle_function, ArrayType(StringType()))
        df = df.withColumn("shingles", shingle_udf(col("tokens")))
        return df

    def hash_shingles(self, df: DataFrame) -> DataFrame:
        def hash_list(shingles):
            hashed = []
            for s in shingles:
                if s and s.strip():
                    h = mmh3.hash64(s.strip(), signed=False)[0] #retourne entier non signé entre 0 et 2^64 -1
                    h = h % (2**63)  # forcer des entier entre 0 et 2^63 -1 #Sinon j'ai eu un overflow pour LongType et donc des valeurs NULL de hashage (mon prob initial)
                    #h = mmh3.hash("some string", signed=False)  # 32-bit unsigned integer
                    hashed.append(h)
            return hashed

        hash_udf = udf(hash_list, ArrayType(LongType()))   #longtype entre - 2^63 et 2^63 -1
        df = df.withColumn("hashed_shingles", hash_udf(col("shingles")))
        return df

    def transform(self, df: DataFrame, text_col: str) -> DataFrame:
        df = self.clean_text(df, text_col)
        df = self.tokenize(df, text_col)
        df = self.generate_shingles(df)
        df = self.hash_shingles(df)
        return df



In [ ]:
data = [
    (1, "Spark is a distributed computing engine for large-scale data processing."),
    (2, "Hadoop is a framework for distributed computing and storage.")
]

df = spark.createDataFrame(data, ["id", "text"])

shingler = Shingling(k=3)
df_result = shingler.transform(df, "text")

df_result.select("id", "shingles", "hashed_shingles").show(truncate=False)


+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |shingles                                                                                                                                                                           |hashed_shingles                                                                                                                                                         |
+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------

In [ ]:
#Version python bof
# import hashlib

# class Shingling:
#     def __init__(self, k=10):
#         """
#         k: length of shingles (in words)
#         """
#         self.k = k

#     def tokenize(self, text):
#         """
#         Simple tokenization: split text into words
#         """
#         return text.lower().split()

#     def generate_shingles(self, tokens):
#         """
#         Generate k-word shingles from token list
#         """
#         shingles = []
#         for i in range(len(tokens) - self.k + 1):
#             shingle = ' '.join(tokens[i:i+self.k])
#             shingles.append(shingle)
#         return shingles

#     def hash_shingle(self, shingle):
#         """
#         Deterministic hash using MD5 truncated to 64 bits
#         """
#         h = int(hashlib.md5(shingle.encode('utf-8')).hexdigest()[:16], 16)
#         h = h % (2**63)  # ensure fits in 64-bit signed integer
#         return h

#     def shingle_document(self, text):
#         """
#         Complete pipeline: tokenize → generate shingles → hash shingles
#         Returns: ordered list of hashed shingles
#         """
#         tokens = self.tokenize(text)
#         shingles = self.generate_shingles(tokens)
#         hashed_shingles = [self.hash_shingle(s) for s in shingles]
#         return hashed_shingles


**Classe CompareSets**

In [ ]:
class CompareSets():
  #méthode statique ici car n'accéde pas à l'attribut de l'objet, n'a besoin que de 2 arguments set1 et set 2
  @staticmethod
  def calculate_jaccard_similarity(set1,set2):
    if not set1 or not set2:
      return 0.0
    s1 = set(set1)
    s2 = set(set2)
    intersection = s1.intersection(s2)
    union = s1.union(s2)
    jaccard_similarity = len(intersection) / len(union)
    return jaccard_similarity


In [ ]:
class MinHashing():
  def __init__(self,n):
    self.n = n
  def hash_set(self,shingle_set):
    n = self.n
    min_i = []
    for i in range(n):
      hashed = []
      for x in shingle_set:
        h = mmh3.hash(str(x), seed=i, signed=False)
        hashed.append(h)
      min_i.append(min(hashed))
    return min_i






In [ ]:
shingles_doc1 = [6840193950337781150, 1695454882927409067, 7334696573069238721]
minhasher = MinHashing(n=5)
signature = minhasher.hash_set(shingles_doc1)

print(signature)
# Output example: [122343123, 2332231, 99221231, 28381223, 112392391]


[1325887520, 1238742759, 137587445, 39641790, 190228959]


In [ ]:
class CompareSignatures():
  @staticmethod
  def compare(sig1,sig2):
    if len(sig1) != len(sig2):
      raise ValueError("Signatures must have the same length")

    matches = sum(1 for a,b in zip(sig1,sig2) if a == b)
    return matches /len(sig1)


In [ ]:
uploaded_files = ["1.txt", "2.txt", "3.txt","4.txt","5.txt","6.txt"]

data = []
for filename in uploaded_files:
    with open(filename, "r", encoding="utf-8") as f:
        text = f.read()
        data.append((filename, text))

df = spark.createDataFrame(data, ["filename", "text"])

# 3️⃣ Appliquer la classe Shingling
shingling = Shingling(k=5)
df_shingles = shingling.transform(df, text_col="text")

# 4️⃣ Voir le résultat
df_shingles.select("filename", "shingles", "hashed_shingles").show(truncate=50)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 9: invalid start byte